#### 0. Data Exploration and Cleaning

In [2]:
# read data
df = spark.read.load("/FileStore/tables/animals_comments_csv-5aaff.gz", format='csv', header = True, inferSchema = True)
df.show(10)

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...|
 ojatro|1031.0| I wanna see what ...|
 Tingle Triggers|1212.0| Well shit now Im ...|
Hope For Paws - O...|1806.0| when I saw the en...|
Hope For Paws - O...|2036.0| Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| Call the teddy Larry|
+--------------------+------+-------------------------------------+
only showing top 10 rows

In [3]:
df.dtypes

Out[92]: [('creator_name', 'string'), ('userid', 'double'), ('comment', 'string')]

In [4]:
#print('There are {} rows in this dataset'.format(df.count()))
print("Number of rows in df:", df.count())


Number of rows in df: 5820035

In [5]:
# Count null values in each columns 

#print('There are {} null values in creator_name column'.format(df.filter(df['creator_name'].isNull()).count()))
#print('There are {} null values in userid column'.format(df.filter(df['userid'].isNull()).count()))
#print('There are {} null values in comment column'.format(df.filter(df['comment'].isNull()).count()))



print('Number of null values in creator_name: ',df.filter(df['creator_name'].isNull()).count())
print('Number of null values in userid: ',df.filter(df['userid'].isNull()).count())
print('Number of null values in comment: ',df.filter(df['comment'].isNull()).count())

Number of null values in creator_name: 32050
Number of null values in userid: 565
Number of null values in comment: 1051

In [6]:
# drop out rows with no comment and no userid

def pre_process(df):
  df_drop = df.filter(df['comment'].isNotNull())
  df_drop = df_drop.filter(df_drop['userid'].isNotNull())
  df_drop = df_drop.dropDuplicates()
  
  print('After dropping, we have ', str(df_drop.count()), 'row in dataframe')
  return df_drop

df_drop = pre_process(df)

After dropping, we have 5757212 row in dataframe

In [7]:
import pyspark.sql.functions as F
#convert text in comment to lower case.
df_clean = df_drop.withColumn('comment', F.lower(F.col('comment')))

In [8]:
df_clean.show()

+-----------------------+--------+------------------------+
 creator_name| userid| comment|
+-----------------------+--------+------------------------+
 Brian Barczyk| 28322.0| since i watched t...|
 WaysideWaifs| 89939.0| why are humans so...|
 TobyTurner|236731.0| roses are red thi...|
 meow meow|260511.0| ..omai-wa.*reject...|
 Brave Wilderness|420830.0| good job on getti...|
꼬부기아빠 My Pet Diary|459543.0|2!!0!!!만명!!!!! 축하...|
 The Dodo|485503.0| who gave thumbs d...|
 Hope For Paws - O...|504270.0| such an intense r...|
 Viktor Larkhill|542233.0| oh that poor cat!!|
 Brave Wilderness|543795.0| you can live your...|
 Emily Rios|548572.0| the editing im sc...|
 Info Marvel|658715.0| quien eres y que ...|
 Live2Thrill|672907.0| do it like u used...|
 Brave Wilderness|711033.0| i have a question...|
 Doug The Pug|713452.0| love ❤️ you|
 Robin Seplut|748458.0| he is handsome. i...|
 The Dodo|765993.0| hunters should be...|
 ViralBe|799211.0| what the fuck is ...|
 Top View Viral|815111.0| oye as uno pero d...|
 Paws Channel|816732.0| i almost pissed m...|
+-----------------------+--------+------------------------+
only showing top 20 rows

##### This is an unlabeled dataset and we want to train a clasifier to identify cat and dog owners. Thus first thing to do is to label each comment. 
1. Label comment when he/she has dogs or cats.   
2. label comment when he/she don't have a dog or cat.   
3. Combine 1 and 2 as our training dataset, and rest of the dataset will be the data we predict.   
4. The strategy to tell if a user own or not own is just using key words (like I have a dog) to tell. Otherwise we can't have better ways and don't have labels.

In [10]:
# find user with preference of dog and cat
# note: please propose your own approach and rule to label data 
cond = (df_clean["comment"].like("%my dog%") | df_clean["comment"].like("%i have a dog%")\
        | df_clean["comment"].like("%my cat%") | df_clean["comment"].like("%i have a cat%") \
        | df_clean["comment"].like("%my dogs%") | df_clean["comment"].like("%my cats%")\
        | df_clean["comment"].like("%my puppies%") | df_clean["comment"].like("%i have dogs%")\
        | df_clean["comment"].like("%i have cats%") | df_clean["comment"].like("%my puppy%")\
        | df_clean["comment"].like("%my kitten%") | df_clean["comment"].like("%i have a puppy%")\
        | df_clean["comment"].like("%i have puppies%"))



df_clean = df_clean.withColumn('dog_cat', cond)

# find user do not have dog or cat


no = (df_clean['comment'].like('%my%') | df_clean['comment'].like('%have%') \
      | df_clean['comment'].like('%my dog%') | df_clean['comment'].like('%my cat%') \
      | df_clean['comment'].like('%my dogs%') | df_clean['comment'].like('%my cats%'))
df_clean = df_clean.withColumn('no_pets', ~no)

In [11]:
df_clean.show()

+-----------------------+--------+------------------------+-------+-------+
 creator_name| userid| comment|dog_cat|no_pets|
+-----------------------+--------+------------------------+-------+-------+
 Brian Barczyk| 28322.0| since i watched t...| false| false|
 WaysideWaifs| 89939.0| why are humans so...| false| true|
 TobyTurner|236731.0| roses are red thi...| false| false|
 meow meow|260511.0| ..omai-wa.*reject...| false| true|
 Brave Wilderness|420830.0| good job on getti...| false| true|
꼬부기아빠 My Pet Diary|459543.0|2!!0!!!만명!!!!! 축하...| false| true|
 The Dodo|485503.0| who gave thumbs d...| false| true|
 Hope For Paws - O...|504270.0| such an intense r...| false| true|
 Viktor Larkhill|542233.0| oh that poor cat!!| false| true|
 Brave Wilderness|543795.0| you can live your...| false| true|
 Emily Rios|548572.0| the editing im sc...| false| true|
 Info Marvel|658715.0| quien eres y que ...| false| true|
 Live2Thrill|672907.0| do it like u used...| false| true|
 Brave Wilderness|711033.0| i have a question...| false| false|
 Doug The Pug|713452.0| love ❤️ you| false| true|
 Robin Seplut|748458.0| he is handsome. i...| false| false|
 The Dodo|765993.0| hunters should be...| false| true|
 ViralBe|799211.0| what the fuck is ...| false| true|
 Top View Viral|815111.0| oye as uno pero d...| false| true|
 Paws Channel|816732.0| i almost pissed m...| false| false|
+-----------------------+--------+------------------------+-------+-------+
only showing top 20 rows

#### 1. Build the classifiter
In order to train a model against the comments, you can use RegexTokenizer to split each comment into a list of words and then use Word2Vec or other model to convert the list to a word vector. What Word2Vec does is to map each word to a unique fixed-size vector and then transform each document into a vector using the average of all words in the document.

In [13]:
from pyspark.ml.feature import RegexTokenizer

regexTokenizer = RegexTokenizer(inputCol = 'comment', outputCol = 'text', pattern = '\\W')
df_clean = regexTokenizer.transform(df_clean)
df_clean.show()

+-----------------------+--------+------------------------+-------+-------+--------------------+
 creator_name| userid| comment|dog_cat|no_pets| text|
+-----------------------+--------+------------------------+-------+-------+--------------------+
 Brian Barczyk| 28322.0| since i watched t...| false| false|[since, i, watche...|
 WaysideWaifs| 89939.0| why are humans so...| false| true|[why, are, humans...|
 TobyTurner|236731.0| roses are red thi...| false| false|[roses, are, red,...|
 meow meow|260511.0| ..omai-wa.*reject...| false| true|[omai, wa, reject...|
 Brave Wilderness|420830.0| good job on getti...| false| true|[good, job, on, g...|
꼬부기아빠 My Pet Diary|459543.0|2!!0!!!만명!!!!! 축하...| false| true| [2, 0]|
 The Dodo|485503.0| who gave thumbs d...| false| true|[who, gave, thumb...|
 Hope For Paws - O...|504270.0| such an intense r...| false| true|[such, an, intens...|
 Viktor Larkhill|542233.0| oh that poor cat!!| false| true|[oh, that, poor, ...|
 Brave Wilderness|543795.0| you can live your...| false| true|[you, can, live, ...|
 Emily Rios|548572.0| the editing im sc...| false| true|[the, editing, im...|
 Info Marvel|658715.0| quien eres y que ...| false| true|[quien, eres, y, ...|
 Live2Thrill|672907.0| do it like u used...| false| true|[do, it, like, u,...|
 Brave Wilderness|711033.0| i have a question...| false| false|[i, have, a, ques...|
 Doug The Pug|713452.0| love ❤️ you| false| true| [love, you]|
 Robin Seplut|748458.0| he is handsome. i...| false| false|[he, is, handsome...|
 The Dodo|765993.0| hunters should be...| false| true|[hunters, should,...|
 ViralBe|799211.0| what the fuck is ...| false| true|[what, the, fuck,...|
 Top View Viral|815111.0| oye as uno pero d...| false| true|[oye, as, uno, pe...|
 Paws Channel|816732.0| i almost pissed m...| false| false|[i, almost, pisse...|
+-----------------------+--------+------------------------+-------+-------+--------------------+
only showing top 20 rows

#### Optional: only select 1000000 rows for testing. (In final version, try to use all dataset)

In [15]:
from pyspark.sql.functions import rand

df_clean.orderBy(rand(seed=0)).createOrReplaceTempView('table')
df_clean = spark.sql("select * from table limit 1000000")

In [16]:
# use word2vec get text vector feature.
from pyspark.ml.feature import Word2Vec

# Learn a mapping from words to Vectors. (choose higher vectorSize here)
word2vec = Word2Vec(vectorSize = 20, minCount = 1, inputCol = 'text', outputCol = 'wordvector')
model = word2vec.fit(df_clean)


df_model = model.transform(df_clean)
df_model.show()

+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+
 creator_name| userid| comment|dog_cat|no_pets| text| wordvector|
+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+
 nywolforg| 472090.0|omg they sleep ju...| true| false|[omg, they, sleep...|[0.11280875490047...|
Hope For Paws - O...|2114449.0|de verdad que soi...| false| true|[de, verdad, que,...|[0.25702872996528...|
 Info Marvel| 185789.0|no creo que metan...| false| true|[no, creo, que, m...|[0.05106595419347...|
 Viktor Larkhill|1589914.0| alive| false| true| [alive]|[0.38546282052993...|
 Caelyn Le|2339310.0|love it. cant wai...| false| true|[love, it, cant, ...|[0.10598695522639...|
 RaleighLink14|1582365.0|i feel like if yo...| false| true|[i, feel, like, i...|[-0.1034655684294...|
 peta2TV| 346689.0| how can i help?!| false| true| [how, can, i, help]|[0.03524387162178...|
 Cole & Marmalade| 723089.0| meowzies!!!| false| true| [meowzies]|[-0.0027790875174...|
 GOOD BYE|2033233.0|beyoncé is so ove...| false| true|[beyonc, is, so, ...|[0.01695460360497...|
 Brave Wilderness|1350178.0|the music makes i...| false| true|[the, music, make...|[0.04145589843392...|
 Brave Wilderness| 321227.0|guys pain toleran...| false| true|[guys, pain, tole...|[0.10847717151045...|
 Robin Seplut| 506166.0| cat. woaw| false| true| [cat, woaw]|[0.00747417518869...|
 Gohan The Husky| 82016.0|do you watch drag...| false| true|[do, you, watch, ...|[-0.2133534584815...|
 Brave Wilderness| 222566.0|you eat meat coyo...| false| true|[you, eat, meat, ...|[-0.0818568185831...|
 Brave Wilderness|1405480.0|hey guys my name ...| false| false|[hey, guys, my, n...|[0.19201903761131...|
 Menthol Kratom|1890992.0|awesome song forsire| false| true|[awesome, song, f...|[-0.1044250829145...|
 Talking Kitty Cat| 482275.0|that kitten looks...| false| false|[that, kitten, lo...|[0.01766259906192...|
 Brian Barczyk| 732406.0|bryan i love the ...| false| false|[bryan, i, love, ...|[0.08498509409172...|
 Gohan The Husky|2370558.0|i think he needs ...| false| true|[i, think, he, ne...|[0.06114463203332...|
 DVLH3|1682878.0| he is so cute| false| true| [he, is, so, cute]|[0.06080232840031...|
+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+
only showing top 20 rows

#### Get training dataset
Note that here I am using training dataset as "Has cat or dog" + "Don't have pets" which is "Dog_cat True" + "No_pets True"

The rest of dataset will be served as dataset for prediction.

In [18]:
df_pets = df_model.filter(F.col('dog_cat') == True)
df_no_pets = df_model.filter(F.col('no_pets') == True)
print('Number of confirmed user who has pets: ', df_pets.count())
print('Number of confirmed user who does not has pets: ', df_no_pets.count())


Number of confirmed user who has pets: 9249
Number of confirmed user who does not has pets: 832784

In [19]:
df_pets.show()

+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+
 creator_name| userid| comment|dog_cat|no_pets| text| wordvector|
+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+
 nywolforg| 472090.0|omg they sleep ju...| true| false|[omg, they, sleep...|[0.11280875490047...|
 Gardea23|1346087.0|hey matt my puppy...| true| false|[hey, matt, my, p...|[0.16277468867170...|
 The Dodo| 48857.0|my dog died yeste...| true| false|[my, dog, died, y...|[0.43401616811752...|
 Vet Ranch|1162567.0|my dog chewy is a...| true| false|[my, dog, chewy, ...|[0.13329727981134...|
 The Dodo| 297832.0|i want them all!t...| true| false|[i, want, them, a...|[0.10117421493799...|
 DarkDynastyK9s| 91432.0|hi im really inte...| true| false|[hi, im, really, ...|[0.06733200748356...|
 Happy Tails|1790011.0|my cats name is r...| true| false|[my, cats, name, ...|[0.15740816099569...|
Stray Rescue of S...| 471458.0|he looks exactly ...| true| false|[he, looks, exact...|[-0.0707124425098...|
 TobyTurner| 953315.0|my cats name was ...| true| false|[my, cats, name, ...|[0.22295317780226...|
 Think Like A Horse| 581653.0|my cat has learne...| true| false|[my, cat, has, le...|[0.13339916302788...|
Worlds Fuzziest V...| 550829.0|0:02 she seems so...| true| false|[0, 02, she, seem...|[0.12306481434087...|
 The Dodo| 505381.0|oooh that hilario...| true| false|[oooh, that, hila...|[0.04760600528162...|
 Vet Ranch|2474101.0|when we rescued m...| true| false|[when, we, rescue...|[0.14110793859830...|
 stacyvlogs|2208271.0|i never vaccinate...| true| false|[i, never, vaccin...|[0.14685671460373...|
 Dogumentary TV| 242960.0|i had a pet germa...| true| false|[i, had, a, pet, ...|[0.18072580300518...|
 Rachel Fusaro| 927355.0|i would bring my ...| true| false|[i, would, bring,...|[0.22180373966693...|
 Taylor Nicole Dean|2047724.0|milo is my cats n...| true| false|[milo, is, my, ca...|[0.06999519301785...|
 Stonnie Dennis|2195639.0|i just love your ...| true| false|[i, just, love, y...|[0.12367917813455...|
 stacyvlogs|1664227.0|dang im really la...| true| false|[dang, im, really...|[0.12032446682918...|
 stacyvlogs| 20526.0|we got my dog fro...| true| false|[we, got, my, dog...|[0.15858964478347...|
+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+
only showing top 20 rows

In [20]:
df_no_pets.show()

+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+
 creator_name| userid| comment|dog_cat|no_pets| text| wordvector|
+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+
Hope For Paws - O...|2114449.0|de verdad que soi...| false| true|[de, verdad, que,...|[0.25702872996528...|
 Info Marvel| 185789.0|no creo que metan...| false| true|[no, creo, que, m...|[0.05106595419347...|
 Viktor Larkhill|1589914.0| alive| false| true| [alive]|[0.38546282052993...|
 Caelyn Le|2339310.0|love it. cant wai...| false| true|[love, it, cant, ...|[0.10598695522639...|
 RaleighLink14|1582365.0|i feel like if yo...| false| true|[i, feel, like, i...|[-0.1034655684294...|
 peta2TV| 346689.0| how can i help?!| false| true| [how, can, i, help]|[0.03524387162178...|
 Cole & Marmalade| 723089.0| meowzies!!!| false| true| [meowzies]|[-0.0027790875174...|
 GOOD BYE|2033233.0|beyoncé is so ove...| false| true|[beyonc, is, so, ...|[0.01695460360497...|
 Brave Wilderness|1350178.0|the music makes i...| false| true|[the, music, make...|[0.04145589843392...|
 Brave Wilderness| 321227.0|guys pain toleran...| false| true|[guys, pain, tole...|[0.10847717151045...|
 Robin Seplut| 506166.0| cat. woaw| false| true| [cat, woaw]|[0.00747417518869...|
 Gohan The Husky| 82016.0|do you watch drag...| false| true|[do, you, watch, ...|[-0.2133534584815...|
 Brave Wilderness| 222566.0|you eat meat coyo...| false| true|[you, eat, meat, ...|[-0.0818568185831...|
 Menthol Kratom|1890992.0|awesome song forsire| false| true|[awesome, song, f...|[-0.1044250829145...|
 Gohan The Husky|2370558.0|i think he needs ...| false| true|[i, think, he, ne...|[0.06114463203332...|
 DVLH3|1682878.0| he is so cute| false| true| [he, is, so, cute]|[0.06080232840031...|
★Amazing Animals TV★| 521129.0|how can you say t...| false| true|[how, can, you, s...|[0.09536402752961...|
 Kitten Academy|1581032.0| i subscribed! :)| false| true| [i, subscribed]|[0.16350548155605...|
 TheNorthwoodsman1|1721605.0|glad you had some...| false| true|[glad, you, had, ...|[0.09293926255777...|
 null|1516010.0|with this episode...| false| true|[with, this, epis...|[0.04419558793306...|
+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+
only showing top 20 rows

Note that number of negative labels is around 100 times more than positive labels, so here we need to downsampling the negative labels. By rule of thumb, the gap should be no more than 10 times. But here I make them balance to the ratio around 1:2 (1 for positive: 2 for negative)

In [22]:
df_no_pets.orderBy(rand()).createOrReplaceTempView('table')

Num_pos_label = df_model.filter(F.col('dog_cat') == True).count()
Num_neg_label = df_model.filter(F.col('no_pets') == True).count()

df_no_pets_down = spark.sql('Select * from table limit {}'.format(Num_pos_label * 2))

In [23]:
df_no_pets_down.show()
print('Number of rows in dataset: ', df_no_pets_down.count())

+-----------------------+---------+----------------------------------+-------+-------+--------------------+--------------------+
 creator_name| userid| comment|dog_cat|no_pets| text| wordvector|
+-----------------------+---------+----------------------------------+-------+-------+--------------------+--------------------+
 Hope For Paws - O...| 999777.0| im just going to ...| false| true|[im, just, going,...|[0.22995067370886...|
 The Elephant Sanc...|2012463.0| more nosey please...| false| true|[more, nosey, ple...|[0.06710431072860...|
 MaxluvsMya| 87743.0| sooooooo freakin ...| false| true|[sooooooo, freaki...|[0.06902813414732...|
 Сергей Детушев|2199545.0| мои после одной а...| false| true| []| (20,[],[])|
 The Dodo|1024834.0| awwwwwwwwwwwwwwww...| false| true|[awwwwwwwwwwwwwww...|[-0.0025001494213...|
 The Dodo| 339687.0| yeah and now ayla...| false| true|[yeah, and, now, ...|[0.07117296134432...|
 Brave Wilderness|1341514.0| who else thought ...| false| true|[who, else, thoug...|[-0.0031166630131...|
 Maymo| 670461.0| mayms...you are t...| false| true|[mayms, you, are,...|[0.27281524347407...|
 Solid Gold Aquatics|2013057.0| wowl thats so coo...| false| true|[wowl, thats, so,...|[-0.0855583245866...|
 Brave Wilderness| 343104.0| before 20k views?| false| true|[before, 20k, views]|[0.08627502123514...|
 Viktor Larkhill|1059692.0| god bless the lit...| false| true|[god, bless, the,...|[0.51749791353940...|
 TCTV|2265851.0| đúng rùi trộm đán...| false| true|[ng, r, i, tr, m,...|[0.39707327801734...|
 Big Cat Rescue|2425427.0| great stuff| false| true| [great, stuff]|[-0.2291133478283...|
 Talking Kitty Cat| 799618.0| oh shelby :) you ...| false| true|[oh, shelby, you,...|[0.06492932617517...|
 Top View Viral|1639340.0| y las mujeres con...| false| true|[y, las, mujeres,...|[0.20238811217925...|
꼬부기아빠 My Pet Diary| 144449.0|원래 고양이를 좋아하지 않았는데...| false| true| []| (20,[],[])|
 Brave Wilderness| 27665.0| need a lot more o...| false| true|[need, a, lot, mo...|[-0.0175281796449...|
 Taylor Nicole Dean|1362314.0| such tiny legs (>...| false| true| [such, tiny, legs]|[-0.0689274420340...|
 Steff J| 288854.0| to cute to be tru...| false| true|[to, cute, to, be...|[0.14406176949185...|
 Brave Wilderness| 76685.0| hi can you get bi...| false| true|[hi, can, you, ge...|[0.00706588733009...|
+-----------------------+---------+----------------------------------+-------+-------+--------------------+--------------------+
only showing top 20 rows

Number of rows in dataset: 18498

In [24]:
print('Now after balancing the lables, we have ')   
print('Positive label: ', Num_pos_label)
print('Negtive label: ', df_no_pets_down.count())

Now after balancing the lables, we have 
Positive label: 9249
Negtive label: 18498

In [25]:
def get_label(df_pets, df_no_pets_down):
  df_labeled = df_pets.select('dog_cat', 'wordvector').union(df_no_pets_down.select('dog_cat','wordvector'))
  return df_labeled

df_labeled = get_label(df_pets, df_no_pets_down)
df_labeled.show()

+-------+--------------------+
dog_cat| wordvector|
+-------+--------------------+
 true|[0.11280875490047...|
 true|[0.16277468867170...|
 true|[0.43401616811752...|
 true|[0.13329727981134...|
 true|[0.10117421493799...|
 true|[0.06733200748356...|
 true|[0.15740816099569...|
 true|[-0.0707124425098...|
 true|[0.22295317780226...|
 true|[0.13339916302788...|
 true|[0.12306481434087...|
 true|[0.04760600528162...|
 true|[0.14110793859830...|
 true|[0.14685671460373...|
 true|[0.18072580300518...|
 true|[0.22180373966693...|
 true|[0.06999519301785...|
 true|[0.12367917813455...|
 true|[0.12032446682918...|
 true|[0.15858964478347...|
+-------+--------------------+
only showing top 20 rows

In [26]:
#convert Boolean value to 1 and 0's
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

def multiple(x):
  return int(x*1)


udf_boolToInt = udf(lambda z: multiple(z), IntegerType())
df_labeled = df_labeled.withColumn('label', udf_boolToInt('dog_cat'))

df_labeled.show()

+-------+--------------------+-----+
dog_cat| wordvector|label|
+-------+--------------------+-----+
 true|[0.11280875490047...| 1|
 true|[0.16277468867170...| 1|
 true|[0.43401616811752...| 1|
 true|[0.13329727981134...| 1|
 true|[0.10117421493799...| 1|
 true|[0.06733200748356...| 1|
 true|[0.15740816099569...| 1|
 true|[-0.0707124425098...| 1|
 true|[0.22295317780226...| 1|
 true|[0.13339916302788...| 1|
 true|[0.12306481434087...| 1|
 true|[0.04760600528162...| 1|
 true|[0.14110793859830...| 1|
 true|[0.14685671460373...| 1|
 true|[0.18072580300518...| 1|
 true|[0.22180373966693...| 1|
 true|[0.06999519301785...| 1|
 true|[0.12367917813455...| 1|
 true|[0.12032446682918...| 1|
 true|[0.15858964478347...| 1|
+-------+--------------------+-----+
only showing top 20 rows

In [27]:
df_labeled_1 = df_labeled.withColumn('label', F.when(df_labeled['dog_cat'] == 'true', 1).otherwise(0)) 
df_labeled_1.show()

+-------+--------------------+-----+
dog_cat| wordvector|label|
+-------+--------------------+-----+
 true|[0.11280875490047...| 1|
 true|[0.16277468867170...| 1|
 true|[0.43401616811752...| 1|
 true|[0.13329727981134...| 1|
 true|[0.10117421493799...| 1|
 true|[0.06733200748356...| 1|
 true|[0.15740816099569...| 1|
 true|[-0.0707124425098...| 1|
 true|[0.22295317780226...| 1|
 true|[0.13339916302788...| 1|
 true|[0.12306481434087...| 1|
 true|[0.04760600528162...| 1|
 true|[0.14110793859830...| 1|
 true|[0.14685671460373...| 1|
 true|[0.18072580300518...| 1|
 true|[0.22180373966693...| 1|
 true|[0.06999519301785...| 1|
 true|[0.12367917813455...| 1|
 true|[0.12032446682918...| 1|
 true|[0.15858964478347...| 1|
+-------+--------------------+-----+
only showing top 20 rows

####Logistic Regression Model

In [29]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

train, test = df_labeled.randomSplit([0.8,0.2], seed = 12345)

lr = LogisticRegression(featuresCol = 'wordvector', labelCol = 'label', maxIter = 10, regParam = 0.1, elasticNetParam = 0.8)

# Run TrainValidationSplit, and choose the best set of parameters
lrmodel = lr.fit(train)



In [30]:
# Make predictions on test data. model is the model with combination of parameters
# that performed best.


predictions =lrmodel.transform(test)
predictions.show()

+-------+--------------------+-----+--------------------+--------------------+----------+
dog_cat| wordvector|label| rawPrediction| probability|prediction|
+-------+--------------------+-----+--------------------+--------------------+----------+
 true|[-0.1602906431071...| 1|[0.80704821763610...|[0.69148014093944...| 0.0|
 true|[-0.1495862051844...| 1|[1.06723935659284...|[0.74407156337873...| 0.0|
 true|[-0.1486990920578...| 1|[0.13828733272383...|[0.53451684412591...| 0.0|
 true|[-0.1256945833563...| 1|[0.74482020699479...|[0.67804900225527...| 0.0|
 true|[-0.1187344744491...| 1|[0.64695126875334...|[0.65632310956295...| 0.0|
 true|[-0.1152730153666...| 1|[1.16259214605539...|[0.76180340217876...| 0.0|
 true|[-0.0966217670356...| 1|[1.04355945174181...|[0.73953622057205...| 0.0|
 true|[-0.0900683429557...| 1|[0.86179846460618...|[0.70303626813927...| 0.0|
 true|[-0.0886675963488...| 1|[0.64361310900783...|[0.65556975152475...| 0.0|
 true|[-0.0807204280324...| 1|[0.46281581596368...|[0.61368195300838...| 0.0|
 true|[-0.0767648554756...| 1|[0.33316921826870...|[0.58253029606550...| 0.0|
 true|[-0.0737984012812...| 1|[0.39394924304273...|[0.59723303656491...| 0.0|
 true|[-0.0689998131245...| 1|[0.52936574097977...|[0.62933516897562...| 0.0|
 true|[-0.0686903089284...| 1|[0.77564809644621...|[0.68474142204462...| 0.0|
 true|[-0.0677239248928...| 1|[0.82239711348725...|[0.69474494371481...| 0.0|
 true|[-0.0675054471939...| 1|[0.91343453732274...|[0.71370246085958...| 0.0|
 true|[-0.0668787904083...| 1|[0.51830910626734...|[0.62675229387595...| 0.0|
 true|[-0.0617712187496...| 1|[0.92049668357548...|[0.71514329759285...| 0.0|
 true|[-0.0604289855808...| 1|[0.90550480023104...|[0.71207942580482...| 0.0|
 true|[-0.0583337329328...| 1|[0.84157826855706...|[0.69879751344736...| 0.0|
+-------+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [31]:
# Extract the summary from the returned LogisticRegressionModel instance trained
#Obtain ROC
trainingsummery = lrmodel.summary
trainingsummery.roc.show()

Exception ignored in: <function JavaWrapper.__del__ at 0x7fece94c2a60>
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/ml/wrapper.py", line 40, in __del__
 if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'RegexTokenizer' object has no attribute '_java_obj'
+--------------------+--------------------+
 FPR| TPR|
+--------------------+--------------------+
 0.0| 0.0|
0.008935824532900082|0.013962315304324251|
0.013200649878147848| 0.03361800189779043|
0.017330083942594096|0.053138132031991324|
0.021527213647441104| 0.0727938186254575|
 0.02552125643108584| 0.09272061813745425|
0.029718386135932844| 0.11278297410871628|
0.033644733279176824| 0.13338755591703944|
 0.03777416734362307| 0.15290768605124033|
0.041091253723260224| 0.17473227599295107|
 0.04515299214730571| 0.1947946319642131|
 0.04982399133495803| 0.21363697980208757|
 0.05388572975900352| 0.2335637793140843|
0.057744381261846736| 0.2537616917446116|
 0.06146764148388844| 0.27409516063440426|
 0.06613864067154075| 0.29253083909448285|
 0.06999729217438397| 0.3125931950657449|
 0.07412672623883022| 0.33224888165921107|
 0.07879772542648253| 0.35068456011928967|
 0.08346872461413485| 0.36939135149789887|
+--------------------+--------------------+
only showing top 20 rows

In [32]:
print('ROC:' + str(trainingsummery.areaUnderROC))

ROC:0.8315083820178504

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

def get_evaluation_result(predictions):
  evaluator = BinaryClassificationEvaluator(
                labelCol = 'label', rawPredictionCol = 'rawPrediction', metricName = 'areaUnderROC')
  AUC = evaluator.evaluate(predictions)
  
  TP = predictions[(predictions['label'] == 1) & (predictions['prediction'] == 1)].count()
  FP = predictions[(predictions['label'] == 0) & (predictions['prediction'] == 1)].count()
  TN = predictions[(predictions['label'] == 0) & (predictions['prediction'] == 0)].count()
  FN = predictions[(predictions['label'] == 1) & (predictions['prediction'] == 0)].count()
  
  
  accuracy = (TP + TN) / (TP + TN + FP + FN)
  precision =  TP / (TP + FP)
  recall = TP / (TP + FN)
  
  
  print ("True Positives:", TP)
  print ("False Positives:", FP)
  print ("True Negatives:", TN)
  print ("False Negatives:", FN)
  print ("Test Accuracy:", accuracy)
  print ("Test Precision:", precision)
  print ("Test Recall:", recall)
  print ("Test AUC of ROC:", AUC)

  
print("Prediction result summary for Logistic Regression Model:  ")
get_evaluation_result(predictions)

Prediction result summary for Logistic Regression Model: 
True Positives: 40
False Positives: 42
True Negatives: 3684
False Negatives: 1832
Test Accuracy: 0.6652375848517328
Test Precision: 0.4878048780487805
Test Recall: 0.021367521367521368
Test AUC of ROC: 0.8213961088860372

####Random Forest Model

In [35]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

# Train a RF model
rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'wordvector', numTrees = 15)

rfmodel = rf.fit(train)
predictions = rfmodel.transform(test)
predictions.show()

+-------+--------------------+-----+--------------------+--------------------+----------+
dog_cat| wordvector|label| rawPrediction| probability|prediction|
+-------+--------------------+-----+--------------------+--------------------+----------+
 true|[-0.1602906431071...| 1|[10.5104633688062...|[0.70069755792041...| 0.0|
 true|[-0.1495862051844...| 1|[10.8398140912165...|[0.72265427274776...| 0.0|
 true|[-0.1486990920578...| 1|[12.2500394538366...|[0.81666929692244...| 0.0|
 true|[-0.1256945833563...| 1|[8.69534414416625...|[0.57968960961108...| 0.0|
 true|[-0.1187344744491...| 1|[8.38653574863840...|[0.55910238324256...| 0.0|
 true|[-0.1152730153666...| 1|[10.8398140912165...|[0.72265427274776...| 0.0|
 true|[-0.0966217670356...| 1|[11.1121315436897...|[0.74080876957931...| 0.0|
 true|[-0.0900683429557...| 1|[10.6885187272559...|[0.71256791515039...| 0.0|
 true|[-0.0886675963488...| 1|[13.1494427688807...|[0.87662951792538...| 0.0|
 true|[-0.0807204280324...| 1|[9.03788622690360...|[0.60252574846024...| 0.0|
 true|[-0.0767648554756...| 1|[9.43110237100214...|[0.62874015806680...| 0.0|
 true|[-0.0737984012812...| 1|[9.43110237100214...|[0.62874015806680...| 0.0|
 true|[-0.0689998131245...| 1|[8.19673034025576...|[0.54644868935038...| 0.0|
 true|[-0.0686903089284...| 1|[11.3395714301149...|[0.75597142867432...| 0.0|
 true|[-0.0677239248928...| 1|[10.6220830000003...|[0.70813886666669...| 0.0|
 true|[-0.0675054471939...| 1|[10.1984701835666...|[0.67989801223777...| 0.0|
 true|[-0.0668787904083...| 1|[7.87634038192547...|[0.52508935879503...| 0.0|
 true|[-0.0617712187496...| 1|[10.6540293930091...|[0.71026862620060...| 0.0|
 true|[-0.0604289855808...| 1|[10.1984701835666...|[0.67989801223777...| 0.0|
 true|[-0.0583337329328...| 1|[10.1984701835666...|[0.67989801223777...| 0.0|
+-------+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [36]:
print('Prediction result summary for Random Forest Model:')
get_evaluation_result(predictions)

Prediction result summary for Random Forest Model:
True Positives: 1198
False Positives: 217
True Negatives: 3509
False Negatives: 674
Test Accuracy: 0.8408360128617364
Test Precision: 0.846643109540636
Test Recall: 0.6399572649572649
Test AUC of ROC: 0.9236874544090727

#### 2. Classify All The Users

we can apply the cat/dog classifier to all the other users in the dataset

In [38]:
df_unknow = df_model.filter((F.col('dog_cat') == False) & (F.col('no_pets') == False))
df_unknow = df_unknow.withColumn('label', df_unknow.dog_cat.cast('integer'))
print("There are {} users whose attribute is unclear.".format(df_unknow.count()))

pred_all = rfmodel.transform(df_unknow)
pred_all.show()

There are 157967 users whose attribute is unclear.
+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 creator_name| userid| comment|dog_cat|no_pets| text| wordvector|label| rawPrediction| probability|prediction|
+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 Brave Wilderness|1405480.0|hey guys my name ...| false| false|[hey, guys, my, n...|[0.19201903761131...| 0|[12.4800186279505...|[0.83200124186337...| 0.0|
 Talking Kitty Cat| 482275.0|that kitten looks...| false| false|[that, kitten, lo...|[0.01766259906192...| 0|[10.1984701835666...|[0.67989801223777...| 0.0|
 Brian Barczyk| 732406.0|bryan i love the ...| false| false|[bryan, i, love, ...|[0.08498509409172...| 0|[11.9193921104537...|[0.79462614069691...| 0.0|
 Brian Barczyk| 63870.0|brian i love ya b...| false| false|[brian, i, love, ...|[0.10848866987541...| 0|[11.7462817515455...|[0.78308545010303...| 0.0|
 Brave Wilderness| 700415.0|isnt he afraid to...| false| false|[isnt, he, afraid...|[-0.0517964592965...| 0|[11.3065033301498...|[0.75376688867665...| 0.0|
 Brian Barczyk| 993751.0|nawww dont worry!...| false| false|[nawww, dont, wor...|[0.24634226900525...| 0|[12.8378460019624...|[0.85585640013083...| 0.0|
Hope For Paws - O...|1534859.0|what a sweet girl...| false| false|[what, a, sweet, ...|[0.15434300138250...| 0|[5.63983052155640...|[0.37598870143709...| 1.0|
 dzlfreek| 653807.0|hey great videos....| false| false|[hey, great, vide...|[0.11556134554882...| 0|[11.6787226083187...|[0.77858150722124...| 0.0|
Deadly Tarantula ...|2041195.0|do you have any s...| false| false|[do, you, have, a...|[-0.1163685242128...| 0|[12.9910413422507...|[0.86606942281671...| 0.0|
 Reptile Channel| 567073.0|i love these but ...| false| false|[i, love, these, ...|[0.12552372195982...| 0|[3.72942801690395...|[0.24862853446026...| 1.0|
 Brave Wilderness| 267784.0|coyote i absolute...| false| false|[coyote, i, absol...|[0.19937540495113...| 0|[13.0361623362932...|[0.86907748908621...| 0.0|
 Brian Barczyk| 4433.0|i cant believe yo...| false| false|[i, cant, believe...|[0.13002196681389...| 0|[11.7062861918032...|[0.78041907945355...| 0.0|
 Loki the Red Fox|1728510.0|awwthats my grand...| false| false|[awwthats, my, gr...|[0.25247142759035...| 0|[10.9451881558992...|[0.72967921039328...| 0.0|
 The Dodo|1362416.0|after watching th...| false| false|[after, watching,...|[0.07801263190185...| 0|[4.70984879772572...|[0.31398991984838...| 1.0|
 Clints Reptiles|2502133.0|would a 55 galon ...| false| false|[would, a, 55, ga...|[-0.1259036170024...| 0|[12.6102437909513...|[0.84068291939675...| 0.0|
 Brian Barczyk|1088286.0|yesterday your wi...| false| false|[yesterday, your,...|[0.11335577638170...| 0|[3.26741780994948...|[0.21782785399663...| 1.0|
 Brave Wilderness|1072622.0|think its to rota...| false| false|[think, its, to, ...|[-0.0225469672431...| 0|[13.2495044495354...|[0.88330029663569...| 0.0|
 TobyTurner| 804839.0|i havent seen you...| false| false|[i, havent, seen,...|[0.03323446223512...| 0|[10.0594890438068...|[0.67063260292045...| 0.0|
 Dan Hiteshew| 520613.0|you ever own any ...| false| false|[you, ever, own, ...|[0.07727923186030...| 0|[9.59045641594803...|[0.63936376106320...| 0.0|
 viralvids|2117802.0|i have no words b...| false| false|[i, have, no, wor...|[0.12001258819630...| 0|[8.94829965810640...|[0.59655331054042...| 0.0|
+--------------------+---------+--------------------+-------+-------+--------------------+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

Fraction of the users who are cat/dog owners (ML estimate):
* Idea is using $$ \frac{\text{Num of owner labeled} + \text{Num of owner predicted}}{ \text{Total users in our used dataset}} $$

In [40]:
# number of total user
total_user = df_model.select('userid').distinct().count()
print('Total user: ',total_user)
#number of labeled owner
owner_labeled = df_pets.select('userid').distinct().count()
print('Total labeled owner: ',owner_labeled)
#number of owner predicted
owner_pred = pred_all.filter(F.col('prediction') == 1).count()
print('Total predicted owner: ',owner_pred)




fraction = (owner_labeled + owner_pred) / total_user
print('Fraction of the users who are cat/dog owners (ML estimate): ', round(fraction,3))


Total user: 687826
Total labeled owner: 8809
Total predicted owner: 38735
Fraction of the users who are cat/dog owners (ML estimate): 0.069

#### 3.Get insights of Users

In [42]:
from pyspark.ml.feature import StopWordsRemover

df_all_owner = df_pets.select('text').union(pred_all.filter(F.col('prediction') == 1).select('text'))

stopwords_custom = ['im', 'get', 'gets', 'got', 'one', 'hes', 'shes', 'he', 'she', 'dog', 'dogs', 'cat', 'cats', 'kitty', 'much', 'really', 'like', 'dont', 'want', 'thin', 'know', 'see', 'also', 'never', 'go', 'ive']

remover1 = StopWordsRemover(inputCol ='raw', outputCol='filtered')
core = remover1.getStopWords()
core = core + stopwords_custom
remover = StopWordsRemover(inputCol='text', outputCol='filtered', stopWords = core)


df_all_owner = remover.transform(df_all_owner)

In [43]:
df_all_owner.show()

+--------------------+--------------------+
 text| filtered|
+--------------------+--------------------+
[omg, they, sleep...| [omg, sleep]|
[hey, matt, my, p...|[hey, matt, puppy...|
[my, dog, died, y...| [died, yesterday]|
[my, dog, chewy, ...|[chewy, older, ve...|
[i, want, them, a...|[therere, cute, o...|
[hi, im, really, ...|[hi, interested, ...|
[my, cats, name, ...| [name, reimi]|
[he, looks, exact...|[looks, exactly, ...|
[my, cats, name, ...| [name]|
[my, cat, has, le...|[learned, climb, ...|
[0, 02, she, seem...|[0, 02, seems, me...|
[oooh, that, hila...|[oooh, hilarious,...|
[when, we, rescue...|[rescued, kitten,...|
[i, never, vaccin...|[vaccinate, need,...|
[i, had, a, pet, ...|[pet, german, she...|
[i, would, bring,...|[bring, home, fri...|
[milo, is, my, ca...|[milo, name, soon...|
[i, just, love, y...|[love, videos, lo...|
[dang, im, really...|[dang, late, want...|
[we, got, my, dog...|[amish, puppy, mi...|
+--------------------+--------------------+
only showing top 20 rows

In [44]:
wc = df_all_owner.select('filtered').rdd.flatMap(lambda x: x.filtered).countByValue()

wcsorted = sorted(wc.items(), key = lambda x: x[1], reverse = True)
wcsorted

Out[128]: [('love', 6422),
 ('time', 4748),
 ('people', 4553),
 ('horse', 3801),
 ('even', 3614),
 ('back', 3550),
 ('little', 3497),
 ('old', 3459),
 ('think', 3350),
 ('day', 3201),
 ('good', 3196),
 ('video', 3159),
 ('years', 3154),
 ('still', 3090),
 ('life', 3086),
 ('2', 2834),
 ('always', 2733),
 ('way', 2633),
 ('take', 2620),
 ('animals', 2594),
 ('first', 2585),
 ('going', 2571),
 ('mom', 2568),
 ('lol', 2524),
 ('put', 2519),
 ('3', 2508),
 ('cant', 2480),
 ('well', 2459),
 ('said', 2406),
 ('around', 2363),
 ('didnt', 2351),
 ('away', 2316),
 ('home', 2277),
 ('help', 2262),
 ('thing', 2248),
 ('right', 2198),
 ('make', 2194),
 ('let', 2188),
 ('house', 2155),
 ('two', 2138),
 ('feel', 2116),
 ('heart', 2108),
 ('name', 2026),
 ('say', 2018),
 ('year', 1993),
 ('bad', 1959),
 ('need', 1949),
 ('died', 1941),
 ('hope', 1923),
 ('thats', 1907),
 ('happy', 1887),
 ('videos', 1885),
 ('great', 1876),
 ('ever', 1866),
 ('baby', 1842),
 ('give', 1840),
 ('every', 1835),
 ('made', 1835),
 ('new', 1805),
 ('getting', 1793),
 ('something', 1789),
 ('better', 1772),
 ('many', 1759),
 ('horses', 1754),
 ('food', 1744),
 ('went', 1733),
 ('oh', 1728),
 ('doesnt', 1726),
 ('care', 1725),
 ('lot', 1708),
 ('keep', 1707),
 ('best', 1664),
 ('family', 1657),
 ('animal', 1655),
 ('us', 1654),
 ('found', 1649),
 ('big', 1642),
 ('pet', 1633),
 ('ago', 1611),
 ('friend', 1571),
 ('watching', 1570),
 ('live', 1563),
 ('god', 1560),
 ('cute', 1536),
 ('since', 1530),
 ('come', 1523),
 ('used', 1516),
 ('saw', 1492),
 ('things', 1484),
 ('u', 1482),
 ('looks', 1470),
 ('bit', 1453),
 ('thank', 1444),
 ('last', 1442),
 ('puppy', 1431),
 ('long', 1426),
 ('look', 1419),
 ('thought', 1418),
 ('1', 1415),
 ('dad', 1410),
 ('work', 1403),
 ('sure', 1377),
 ('someone', 1370),
 ('favorite', 1345),
 ('sad', 1311),
 ('please', 1300),
 ('months', 1293),
 ('poor', 1264),
 ('another', 1263),
 ('hard', 1260),
 ('started', 1252),
 ('days', 1245),
 ('took', 1244),
 ('though', 1244),
 ('head', 1243),
 ('eat', 1235),
 ('4', 1223),
 ('5', 1218),
 ('girl', 1212),
 ('wish', 1199),
 ('use', 1194),
 ('find', 1193),
 ('sorry', 1188),
 ('watch', 1177),
 ('loves', 1170),
 ('came', 1170),
 ('stop', 1153),
 ('trying', 1147),
 ('actually', 1147),
 ('tell', 1141),
 ('water', 1140),
 ('told', 1130),
 ('man', 1130),
 ('makes', 1117),
 ('kitten', 1113),
 ('snake', 1113),
 ('face', 1101),
 ('try', 1093),
 ('almost', 1092),
 ('lost', 1091),
 ('times', 1087),
 ('friends', 1085),
 ('wanted', 1084),
 ('anything', 1062),
 ('everything', 1057),
 ('seen', 1054),
 ('school', 1054),
 ('vet', 1049),
 ('beautiful', 1046),
 ('eyes', 1026),
 ('hand', 1026),
 ('cage', 1002),
 ('thanks', 994),
 ('black', 994),
 ('wont', 987),
 ('maybe', 985),
 ('without', 960),
 ('pain', 952),
 ('person', 949),
 ('guy', 946),
 ('left', 942),
 ('night', 942),
 ('cause', 939),
 ('outside', 939),
 ('hurt', 934),
 ('play', 933),
 ('tried', 924),
 ('today', 920),
 ('named', 917),
 ('scared', 916),
 ('small', 914),
 ('next', 912),
 ('week', 911),
 ('world', 905),
 ('youre', 902),
 ('loved', 899),
 ('amazing', 898),
 ('enough', 880),
 ('hate', 880),
 ('parents', 880),
 ('probably', 880),
 ('show', 877),
 ('omg', 872),
 ('weeks', 861),
 ('fish', 861),
 ('mean', 857),
 ('10', 850),
 ('id', 850),
 ('gave', 850),
 ('whole', 848),
 ('done', 848),
 ('channel', 845),
 ('nothing', 843),
 ('couldnt', 842),
 ('place', 841),
 ('run', 838),
 ('room', 831),
 ('sometimes', 831),
 ('may', 831),
 ('wrong', 828),
 ('ok', 816),
 ('brother', 816),
 ('able', 814),
 ('hamster', 814),
 ('anyone', 808),
 ('snakes', 804),
 ('pretty', 804),
 ('ill', 803),
 ('called', 800),
 ('glad', 799),
 ('feed', 797),
 ('already', 793),
 ('part', 791),
 ('6', 785),
 ('understand', 785),
 ('wouldnt', 774),
 ('end', 773),
 ('kittens', 772),
 ('wasnt', 770),
 ('looking', 769),
 ('believe', 769),
 ('kind', 768),
 ('leave', 764),
 ('sweet', 764),
 ('sister', 764),
 ('pets', 763),
 ('walk', 763),
 ('might', 763),
 ('theyre', 763),
 ('sick', 760),
 ('

#### 4.Identify Creators With Cat And Dog Owners In The Audience
Simply filter out all dog and cat owners and aggregate the creators.

In [46]:
df_create = df_pets.select('creator_name').union(pred_all.filter(F.col('prediction') == 1).select('creator_name'))

df_create.createOrReplaceTempView('create_table')

#get counts

create_count = spark.sql("select distinct creator_name, count(*) as Number \
                          from create_table \
                          group by creator_name \
                          order by Number Desc")


In [47]:
create_count.show()

+--------------------+------+
 creator_name|Number|
+--------------------+------+
 Brave Wilderness| 4980|
 The Dodo| 3729|
 Taylor Nicole Dean| 2953|
 Brian Barczyk| 2490|
Hope For Paws - O...| 1460|
 Think Like A Horse| 1297|
 Vet Ranch| 1221|
 Cole & Marmalade| 1210|
 RaleighLink14| 1205|
 Gohan The Husky| 1202|
 Robin Seplut| 1007|
 Viktor Larkhill| 990|
 stacyvlogs| 987|
Zak Georges Dog T...| 904|
Gone to the Snow ...| 849|
 ViralHog| 604|
 Talking Kitty Cat| 561|
 HammyLux| 501|
 Mr. Max T.V.| 445|
 meow meow| 414|
+--------------------+------+
only showing top 20 rows

'Brave Wilderness', 'The Dodo' and 'Taylor Nicole Dean' are the top three creators with largest distinct cat or dog owner audience population